In [1]:
import numpy as np
import pandas as pd

### Read Data

In [2]:
# read a small number of rows
df = pd.read_csv("datasets/weather_underground.csv", nrows=2)
df

,date,maxpressurem,maxdewptm,maxpressurei,maxdewpti,since1julheatingdegreedaysnormal,heatingdegreedaysnormal,since1sepcoolingdegreedaysnormal,hail,since1julsnowfallm,...,precipi,snowfalli,since1jancoolingdegreedaysnormal,precipm,snowfallm,thunder,monthtodateheatingdegreedays,meantempi,maxvism,meantempm
0,2011-05-01,1026,6,30.31,42,4646,8,NaN,0,157.23,...,0.0,0.0,13,0.0,0.0,0,5,60,16,16
1,2011-05-02,1026,10,30.31,50,4653,7,NaN,0,157.23,...,0.0,0.0,14,0.0,0.0,0,13,57,16,14


In [3]:
# take a subset of columns
cols = df.columns.tolist()
cols[:4]

['date', 'maxpressurem', 'maxdewptm', 'maxpressurei']

In [4]:
# read selected columns, limited rows, and cast data types
dtype = {'date' : str, 'maxpressurem': int, 'maxdewptm': float, 'maxpressurei': float}
df = pd.read_csv("datasets/weather_underground.csv", nrows=2, usecols=cols[:4], dtype=dtype)
df

,date,maxpressurem,maxdewptm,maxpressurei
0,2011-05-01,1026,6.0,30.31
1,2011-05-02,1026,10.0,30.31


In [5]:
# check data type has been properly casted
df.maxdewptm

0     6.0
1    10.0
Name: maxdewptm, dtype: float64

### Data Wrangling

In [6]:
# select subset of columns by data types
df.select_dtypes(include=['object', 'int'])

,date,maxpressurem
0,2011-05-01,1026
1,2011-05-02,1026


In [7]:
# map categorical variables
level_map = {'2011-05-01': 'day 1', '2011-05-02': 'day 2'}
df['cat_level'] = df['date'].map(level_map)
df

,date,maxpressurem,maxdewptm,maxpressurei,cat_level
0,2011-05-01,1026,6.0,30.31,day 1
1,2011-05-02,1026,10.0,30.31,day 2


In [8]:
# map numeric variables
def rule(x):
    if x < 7: return 'low'
    return 'high'

df['cut_level'] = df.apply(lambda x: rule(x['maxdewptm']), axis =  1)
df

,date,maxpressurem,maxdewptm,maxpressurei,cat_level,cut_level
0,2011-05-01,1026,6.0,30.31,day 1,low
1,2011-05-02,1026,10.0,30.31,day 2,high


In [9]:
# aggregate columns
df['maximum'] = df[['maxdewptm','maxpressurei']].max(axis =1)
df

,date,maxpressurem,maxdewptm,maxpressurei,cat_level,cut_level,maximum
0,2011-05-01,1026,6.0,30.31,day 1,low,30.31
1,2011-05-02,1026,10.0,30.31,day 2,high,30.31


In [10]:
df = pd.read_csv("datasets/weather_underground.csv", usecols=cols[:4])
df.head()

,date,maxpressurem,maxdewptm,maxpressurei
0,2011-05-01,1026,6,30.31
1,2011-05-02,1026,10,30.31
2,2011-05-03,1021,15,30.14
3,2011-05-04,1017,14,30.03
4,2011-05-05,1016,4,30.01


In [11]:
# select certain rows
# df_filter is a boolean mask
df_filter = df['date'].isin(['2011-05-04','2011-05-05'])
df[df_filter]

,date,maxpressurem,maxdewptm,maxpressurei
3,2011-05-04,1017,14,30.03
4,2011-05-05,1016,4,30.01


In [12]:
# smallest partition is numbered highest
cut_points = [np.percentile(df['maxdewptm'], i) for i in [25, 50, 75]]
print(cut_points)

df['group'] = 1
for i in range(3):
    df['group'] = df['group'] + (df['maxdewptm'] < cut_points[i])
df.head()

[10.0, 14.5, 17.75]


,date,maxpressurem,maxdewptm,maxpressurei,group
0,2011-05-01,1026,6,30.31,4
1,2011-05-02,1026,10,30.31,3
2,2011-05-03,1021,15,30.14,2
3,2011-05-04,1017,14,30.03,3
4,2011-05-05,1016,4,30.01,4


In [13]:
# save without storing numeric index
df.to_csv("datasets/weather_underground_subset.csv", index=None)

#### NULL Handling

In [14]:
# count nulls in each rows
df = pd.DataFrame({ 'id': [1,2,3], 'c1':[0,0,np.nan], 'c2': [np.nan,1,1]})
df = df[['id', 'c1', 'c2']]
df['num_nulls'] = df[['c1', 'c2']].isnull().sum(axis=1)
df.head()

,id,c1,c2,num_nulls
0,1,0.0,NaN,1
1,2,0.0,1.0,0
2,3,NaN,1.0,1


In [15]:
df.shape

(3, 4)

In [16]:
# drop rows that contains any null
df.dropna().shape

(1, 4)

In [17]:
# drop rows that have nulls in specific columns
df.dropna(subset=['c2']).shape

(2, 4)